# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234830606926                   -0.50    7.0         
  2   -7.250223203944       -1.81       -1.39    1.0   5.18ms
  3   -7.250907784228       -3.16       -1.89    1.0   5.32ms
  4   -7.250894189228   +   -4.87       -1.84    2.0   6.53ms
  5   -7.251334660877       -3.36       -2.76    1.0   5.40ms
  6   -7.251338148692       -5.46       -3.22    1.0   5.67ms
  7   -7.251338640428       -6.31       -3.41    2.0   7.00ms
  8   -7.251338785279       -6.84       -3.95    1.0   5.69ms
  9   -7.251338796339       -7.96       -4.33    3.0   7.76ms
 10   -7.251338798400       -8.69       -4.89    1.0   5.86ms
 11   -7.251338798673       -9.56       -5.28    3.0   7.60ms
 12   -7.251338798701      -10.55       -5.77    1.0   6.02ms
 13   -7.251338798704      -11.55       -6.30    2.0   6.91ms
 14   -7.251338798705      -12.38       -6.83    2.0   7.47ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05741606623698882
[ Info: Arnoldi iteration step 2: normres = 0.6121416047785694
[ Info: Arnoldi iteration step 3: normres = 0.8381029034718417
[ Info: Arnoldi iteration step 4: normres = 0.32372800423183584
[ Info: Arnoldi iteration step 5: normres = 0.4261870717537284
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.61e-02, 6.89e-02, 2.83e-01, 3.08e-01, 2.02e-02)
[ Info: Arnoldi iteration step 6: normres = 0.31186515583201846
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (9.75e-03, 1.12e-01, 2.64e-01, 7.86e-02, 5.20e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06505640786767604
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.02e-04, 7.76e-03, 1.37e-02, 1.94e-02, 5.48e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10302511701615821
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.33e